In [1]:
import numpy as np
from TDD.TDD import Ini_TDD,Clear_TDD,set_index_order,get_unique_table_num,set_root_of_unit,get_count,cont,renormalize
from TDD.TDD_Q import cir_2_tn,get_real_qubit_num,add_trace_line,add_inputs,add_outputs,gen_cir
from TDD.TN import Index,Tensor,TensorNetwork
import time
import random
from qiskit import QuantumCircuit
import cProfile

In [611]:
def simulate(cir,ini=False):
    tn,all_indexs = cir_2_tn(cir)
    tn.tensors=[ts for ts in tn.tensors if ts.name!='nu_q']
#     A=[ts.name for ts in tn.tensors]
#     print(A)
    if ini:
        var=[]
        for idx in all_indexs:
            if idx[0]=='x' and not '_' in idx:
                var.append('a'+idx[1:])
            var.append(idx)
            if idx[0]=='y' and not '_' in idx:
                var.append('b'+idx[1:])
#         print(var)
        for n in range(5):
            for k in range(50):
                s ='x'+str(n)+'_'+str(k)
                if not s in var:
                    var.append(s)
                s ='y'+str(n)+'_'+str(k)
                if not s in var:
                    var.append(s)               
                
        
        Ini_TDD(index_order=var)
        set_root_of_unit(2**3)

#     t_start=time.time()
    tdd=tn.cont()
#     print(time.time()-t_start)
#     print(tdd.weight)
#     print(tdd.node_number())
    return tdd

def change_index(tdd,x,y):
    """change the index from x to y"""
    
    idx_set = []
    key_2_index  = dict()
    index_2_key = dict()
    
    for idx in tdd.index_set:
        if idx.key[0]==x:
            idx_set.append(Index(y+idx.key[1:],idx.idx))
        else:
            idx_set.append(idx)
    
    for k in tdd.key_2_index:
        if isinstance(tdd.key_2_index[k],str) and tdd.key_2_index[k][0]==x:
            key_2_index[k]=y+tdd.key_2_index[k][1:]
        else:
            key_2_index[k]=tdd.key_2_index[k]
            
    for k in tdd.index_2_key:
        if isinstance(k,str) and k[0]==x:
            index_2_key[y+k[1:]]=tdd.index_2_key[k]
        else:
            index_2_key[k]=tdd.index_2_key[k]
    tdd.index_set=idx_set
    tdd.key_2_index=key_2_index
    tdd.index_2_key = index_2_key
    
    
def update_tdd(tdd,cir_head,cir_end):
#     cir_head.x([0,1,2])
#     cir_head.x([0,1,2])
#     cir_end.x([0,1,2])
#     cir_end.x([0,1,2])
    tdd1 = simulate(cir_head)
    change_index(tdd1,'x','a')
    change_index(tdd1,'y','x')

    tdd2 = simulate(cir_end)
    change_index(tdd2,'y','b')
    change_index(tdd2,'x','y')
    
    tdd_new=cont(tdd1,tdd)
    tdd_new=cont(tdd_new,tdd2)

    change_index(tdd_new,'a','x')
    change_index(tdd_new,'b','y')
    # tdd_new=renormalize(tdd_new)
    return tdd_new

def eliminate_control2(tdd,mode='x'):
    
    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)

    if mode=='x':
        x=tdd.node.key
        the_map = tdd.node.out_maps[1]
    else:
        x=tdd.node.successor[0].key
        the_map = tdd.node.successor[0].out_maps[1]
    
    q_c = int(tdd.key_2_index[x][1:])

    flag = False
    if the_map.level==-1:
        return tdd.self_copy(),cir_head,cir_end
    
    while the_map.level>-1:
        if x%2!=the_map.level%2:
            the_map=the_map.father
        else:
            flag = True
            temp_map=the_map
            the_map=the_map.father

    if flag == False:
        return tdd.self_copy(),cir_head,cir_end
    
    q = int(tdd.key_2_index[temp_map.level][1:])

    if tdd.key_2_index[temp_map.level][0]=='x':
        if temp_map.rotate>0:
            cir_head.cp(np.pi/4*(8-temp_map.rotate),q_c,q)
        if temp_map.x==1:
            cir_head.cx(q_c,q)
    else:
        if temp_map.x==1:
            cir_end.cx(q_c,q)
        if temp_map.rotate>0:
            cir_end.cp(np.pi/4*(8-temp_map.rotate),q_c,q)
    # print(cir_head.qasm())
    # print(cir_end.qasm())
    tdd_new = update_tdd(tdd,cir_head,cir_end)
    tdd_new2,cir_head2,cir_end2 = eliminate_control2(tdd_new,mode)
    return tdd_new2,cir_head2&cir_head,cir_end&cir_end2
    
def eliminate_top_control2(tdd):
    tdd_new,cir_head,cir_end = eliminate_control2(tdd,'x')
    
    tdd_new2,cir_head2,cir_end2 = eliminate_control2(tdd_new,'y')
    

    if tdd_new2.map.level>-1:
        tdd_new3,cir_head3,cir_end3=eliminate_global_map(tdd_new2)
        return tdd_new3,cir_head3&cir_head2&cir_head,cir_end&cir_end2&cir_end3
    
    
    return tdd_new2,cir_head2&cir_head,cir_end&cir_end2


def eliminate_top_partial_control2(tdd):
    
    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)

    the_map = tdd.node.out_maps[1]
    if the_map.father.level==-1:
        the_map = tdd.node.successor[0].out_maps[1]
    else:
        the_map = the_map.father
    
    q_c = 0
    
    if the_map.level==-1:
        return tdd.self_copy(),cir_head,cir_end

    while the_map.level>-1:
        temp_map=the_map
        the_map=the_map.father
    
    q = int(tdd.key_2_index[temp_map.level][1:])

    if tdd.key_2_index[temp_map.level][0]=='x':
        if temp_map.rotate>0:
            cir_head.cp(np.pi/4*(8-temp_map.rotate),q_c,q)
        if temp_map.x==1:
            cir_head.cx(q_c,q)
    else:
        if temp_map.x==1:
            cir_end.cx(q_c,q)
        if temp_map.rotate>0:
            cir_end.cp(np.pi/4*(8-temp_map.rotate),q_c,q)

    tdd_new = update_tdd(tdd,cir_head,cir_end)
    tdd_new2,cir_head2,cir_end2 = eliminate_top_partial_control2(tdd_new)
    
    if tdd_new2.map.level>-1:
        tdd_new3,cir_head3,cir_end3=eliminate_global_map(tdd_new2)
        return tdd_new3,cir_head3&cir_head2&cir_head,cir_end&cir_end2&cir_end3
    
    return tdd_new2,cir_head2&cir_head,cir_end&cir_end2

In [612]:
def eliminate_global_map(tdd):
    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)


    if tdd.map.level==-1:
        return tdd.self_copy(),cir_head,cir_end

    the_map=tdd.map
    while the_map.father.level>-1:
        the_map=the_map.father
        
    q = int(tdd.key_2_index[the_map.level][1:])
    if tdd.key_2_index[the_map.level][0]=='x':
        if the_map.rotate>0:
            cir_head.p(np.pi/4*(8-the_map.rotate),q)
        if the_map.x==1:
            cir_head.x(q)
    else:
        if the_map.x==1:
            cir_end.x(q)
        if the_map.rotate>0:
            cir_end.p(np.pi/4*(8-the_map.rotate),q)
    tdd_new = update_tdd(tdd,cir_head,cir_end)
    tdd_new2,cir_head2,cir_end2 = eliminate_global_map(tdd_new)
    return tdd_new2,cir_head2&cir_head,cir_end&cir_end2


def eliminate_control(tdd,x,the_map):
    
    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)

    q_c = int(tdd.key_2_index[x][1:])
    
    while the_map.level>-1:
        if x%2!=the_map.level%2:
            the_map=the_map.father
            continue
        q = int(tdd.key_2_index[the_map.level][1:])
#         print(q_c,q)

        if tdd.key_2_index[the_map.level][0]=='x':
            if the_map.rotate>0:
                cir_head.cp(np.pi/4*(8-the_map.rotate),q_c,q)
            if the_map.x==1:
                cir_head.cx(q_c,q)                
        else:
            if the_map.x==1:
                cir_end.cx(q_c,q)
            if the_map.rotate>0:
                cir_end.cp(np.pi/4*(8-the_map.rotate),q_c,q)
        the_map=the_map.father
        
    tdd_new = update_tdd(tdd,cir_head,cir_end)
    tdd_new = renormalize(tdd_new)
    return tdd_new,cir_head,cir_end


def eliminate_top_control(tdd):
    the_map1 = tdd.node.out_maps[1]
    tdd_new,cir_head,cir_end = eliminate_control(tdd,tdd.node.key,the_map1)
    
    the_map2 = tdd.node.successor[0].out_maps[1]
    tdd_new2,cir_head2,cir_end2 = eliminate_control(tdd_new,tdd.node.successor[0].key,the_map2)
    

    if tdd_new2.map.level>-1:
        tdd_new3,cir_head3,cir_end3=eliminate_global_map(tdd_new2)
        return tdd_new3,cir_head3&cir_head2&cir_head,cir_end&cir_end2&cir_end3
    
    
    return tdd_new2,cir_head2&cir_head,cir_end&cir_end2


def reduce_hadamard2(tdd,n,qc,qt,case=0):
    # n = 2
    # cir_head = QuantumCircuit(n)
    # cir_end = QuantumCircuit(n)
    

    if case<2:
        k=case
        a=bin(k+1)[2:]
        a='0'*(2-len(a))+a
        cir_head = QuantumCircuit(n)
        cir_end = QuantumCircuit(n)
        tdd_temp=tdd.self_copy()
        if a[1]=='0':
            cir_head.cz(qc,qt)
            cir_head.h(qc)
            cir_head.h(qt)
            cir_head.cz(qc,qt)
        if a[0]=='0':
            cir_end.cz(qc,qt)
            cir_end.h(qc)
            cir_end.h(qt)
            cir_end.cz(qc,qt)
        
        tdd_new = update_tdd(tdd_temp,cir_head,cir_end)
        return tdd_new,cir_head,cir_end
    else:
        k=case-2
        a=bin(k+1)[2:]
        a='0'*(2-len(a))+a
        cir_head = QuantumCircuit(n)
        cir_end = QuantumCircuit(n)
        tdd_temp=tdd.self_copy()
        if a[1]=='0':
            cir_head.cz(qc,qt)
            cir_head.h(qc)
            cir_head.h(qt)
            cir_head.cz(qc,qt)
            cir_head.h(qc)
            cir_head.h(qt)
            cir_head.cz(qc,qt)            
        if a[0]=='0':
            cir_end.cz(qc,qt)
            cir_end.h(qc)
            cir_end.h(qt)
            cir_end.cz(qc,qt)
            cir_end.h(qc)
            cir_end.h(qt)
            cir_end.cz(qc,qt)            
        
        tdd_new = update_tdd(tdd_temp,cir_head,cir_end)
        return tdd_new,cir_head,cir_end

def idx_2_qubit(tdd,idx):
    if idx==-1:
        return 100000
    return int(tdd.key_2_index[idx][1:])

def get_max_qubit(tdd,q_limit=-1):
    the_map1 = tdd.node.out_maps[1]
    if the_map1.level==tdd.node.key-1:
        the_map1=the_map1.father
    the_map2 = tdd.node.successor[0].out_maps[1]

    if the_map1.level==-1 or the_map2.level==-1:
        return 100000
    
    while idx_2_qubit(tdd,the_map1.father.level)<q_limit:
        the_map1=the_map1.father
    while idx_2_qubit(tdd,the_map2.father.level)<q_limit:
        the_map2=the_map2.father
    # print(idx_2_qubit(tdd,the_map1.level),the_map1.level,idx_2_qubit(tdd,the_map2.level),the_map2.level,q_limit)
    if max(idx_2_qubit(tdd,the_map1.level),idx_2_qubit(tdd,the_map2.level))<q_limit:
        return max(idx_2_qubit(tdd,the_map1.level),idx_2_qubit(tdd,the_map2.level))
    else:
        return min(idx_2_qubit(tdd,the_map1.level),idx_2_qubit(tdd,the_map2.level))

def eliminate_top_entangle(tdd,q_limit=100000):

    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)
    
    the_map1 = tdd.node.out_maps[1]
    if the_map1.level==tdd.node.key-1:
        the_map1=the_map1.father
    
    the_map2 = tdd.node.successor[0].out_maps[1]
    if the_map1.level==-1 or the_map2.level==-1:
        return tdd.self_copy(),cir_head,cir_end

    q_c = int(tdd.key_2_index[tdd.node.key][1:])
    q_t = get_max_qubit(tdd,q_limit)
    print(q_t,q_limit)
    if q_t == q_limit:
        return tdd.self_copy(),cir_head,cir_end
        
    flag = False
    for k in range(4):
        tdd_new,cir_head,cir_end = reduce_hadamard2(tdd,n,q_c,q_t,k)
        if get_max_qubit(tdd_new,q_limit)!=q_t:
            flag = True
            break
    
    if not flag:
        tdd_new = tdd.self_copy()
        cir_head = QuantumCircuit(n)
        cir_end = QuantumCircuit(n)
    
    tdd_new2,cir_head2,cir_end2 = eliminate_top_entangle(tdd_new,q_t)
    
    return tdd_new2,cir_head2&cir_head,cir_end&cir_end2

def eliminate_top_matrix(tdd):
    
    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)

    M = np.zeros([2,2])
    M[0][0]=M[0][1]=1
    a = tdd.node.successor[0].out_weight[1]
    M[1][0] = a
    
    the_map = tdd.node.out_maps[1]
    if the_map.rotate>0:
        a*=np.exp(1j*np.pi/4*(the_map.rotate))
        M[1][1]=a
    if the_map.x==1:
        M[0][1],M[1][1] = M[1][1],M[0][1]

    M/= np.sqrt(abs(M[0][0])**2+abs(M[0][1])**2)
    M=M.T.conjugate()
    print(M)
    q = int(tdd.key_2_index[tdd.node.key][1:])
    if the_map.father.level==-1:
        cir_head.unitary(M,q)
    else:
        cir_end.unitary(M,q)

    
    tdd_new = update_tdd(tdd,cir_head,cir_end)
    return tdd_new,cir_head,cir_end
    
def eliminate_top_partial_control(tdd):
    n = int(tdd.key_2_index[0][1:])+1
    cir_head = QuantumCircuit(n)
    cir_end = QuantumCircuit(n)

    q_c = 0
    
    the_map = tdd.node.out_maps[1]
    if the_map.father.level==-1:
        the_map = tdd.node.successor[0].out_maps[1]
    else:
        the_map = the_map.father
     
    while the_map.level>-1:

        q = int(tdd.key_2_index[the_map.level][1:])

        if q_c==q: print(the_map)

        if tdd.key_2_index[the_map.level][0]=='x':
            if the_map.rotate>0:
                cir_head.cp(np.pi/4*(8-the_map.rotate),q_c,q)
            if the_map.x==1:
                cir_head.cx(q_c,q)
        else:
            if the_map.x==1:
                cir_end.cx(q_c,q)
            if the_map.rotate>0:
                cir_end.cp(np.pi/4*(8-the_map.rotate),q_c,q)
        the_map=the_map.father
        
    tdd_new = update_tdd(tdd,cir_head,cir_end)
    tdd_new = renormalize(tdd_new)
    return tdd_new,cir_head,cir_end

def eliminate_top_level(tdd):
    
    tdd_new,cir_head,cir_end=eliminate_global_map(tdd)
    tdd_new2,cir_head2,cir_end2=eliminate_top_control2(tdd_new)
    tdd_new3,cir_head3,cir_end3=eliminate_top_entangle(tdd_new2)
    tdd_new4,cir_head4,cir_end4 = eliminate_top_matrix(tdd_new3)
    # tdd_new = renormalize(tdd_new)
    tdd_new5,cir_head5,cir_end5=eliminate_top_partial_control2(tdd_new4)
    # return tdd_new4,cir_head4&cir_head3&cir_head2&cir_head,cir_end&cir_end2&cir_end3&cir_end4
    return tdd_new5,cir_head5&cir_head4&cir_head3&cir_head2&cir_head,cir_end&cir_end2&cir_end3&cir_end4&cir_end5
    
def syn(tdd):
    tdd_new,cir_head,cir_end = eliminate_top_level(tdd)
    # tdd_new,cir_head,cir_end=eliminate_global_map(tdd)
    # tdd_new2,cir_head2,cir_end2=eliminate_top_control(tdd_new)
    # tdd_new2,cir_head2,cir_end2=eliminate_top_level(tdd_new)
    
    # return tdd_new3,cir_head3&cir_head2&cir_head,cir_end&cir_end2&cir_end3
    # return tdd_new2,cir_head2&cir_head,cir_end&cir_end2
    return tdd_new,cir_head,cir_end
    

In [613]:
# n=3
# m=15

# for k in range(100):
#     cir=gen_cir('Random_Clifford_T',n,m)
#     tdd=simulate(cir,True)
#     tdd_new2,cir_head2,cir_end2=syn(tdd)
#     if tdd_new2.map.level>-1:
#         print('find one')
#         break
#     if k%30==0 and k>0:
#         print(k)
# # tdd.show(False)
# # tdd_new,cir_head,cir_end=syn(tdd)

# # tdd_new.show(False)
# cir.draw()

In [ ]:
n=4
m=20

cir=gen_cir('Random_Clifford',n,m)
tdd=simulate(cir,True)
tdd_new,cir_head,cir_end=syn(tdd)
tdd_new.show(False)
# cir_end.draw()

3 100000
2 3
1 2
1 1
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]


C:\Users\23626\AppData\Local\Temp\ipykernel_8852\1848040228.py:200: ComplexWarning: Casting complex values to real discards the imaginary part
  M[1][0] = a
C:\Users\23626\AppData\Local\Temp\ipykernel_8852\1848040228.py:205: ComplexWarning: Casting complex values to real discards the imaginary part
  M[1][1]=a


In [ ]:
# cir.draw()
# tdd_new5,cir_head5,cir_end5=eliminate_top_control(tdd_new4)
# tdd_new2,cir_head,cir_end=eliminate_top_control(tdd_new)
# tdd_new,cir_head,cir_end=syn(tdd_new)
# tdd_new.show(False)

In [528]:
# f=open('cannot be reduced by h2.qasm','w')
# f.write(cir.qasm())
# f.close()